# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [18]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from nltk.tokenize import word_tokenize, RegexpTokenizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
import pickle
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AB\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AB\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AB\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('Disaster', con=engine)
# drop coulmn with null values
df = df[~(df.isnull().any(axis=1))|((df.original.isnull())&~(df.offer.isnull()))]
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Features 
X= df['message'].values

#Target 
y=df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    # remove all punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = word_tokenize(text)
    
    # LEMMATIZING
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
def model_pipeline():
    modelclassifier=MultiOutputClassifier(RandomForestClassifier())
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', modelclassifier)
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:

# spilliting data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [76]:
y_pred = model.predict(X_test)

In [78]:
Accuracy_overall=(y_pred == y_test).mean().mean()
print('Avg Accuracy Overall {0:.3f}% \n'.format(Accuracy_overall*100))

Avg Accuracy Overall 92.226% 



In [82]:
y_pre = pd.DataFrame(y_pred, columns = y_test.columns)
for i in y_test.columns:
    print('_______________________________________________________________\n\n')
    print('FEATURES: {}\n'.format(i))
    print(classification_report(y_test[i],y_pre[i]))

_______________________________________________________________


FEATURES: related

              precision    recall  f1-score   support

           0       0.36      0.12      0.18      1247
           1       0.76      0.93      0.84      3953
           2       0.00      0.00      0.00        42

    accuracy                           0.73      5242
   macro avg       0.38      0.35      0.34      5242
weighted avg       0.66      0.73      0.68      5242

_______________________________________________________________


FEATURES: request

              precision    recall  f1-score   support

           0       0.83      0.98      0.90      4310
           1       0.44      0.06      0.10       932

    accuracy                           0.82      5242
   macro avg       0.64      0.52      0.50      5242
weighted avg       0.76      0.82      0.76      5242

_______________________________________________________________


FEATURES: offer

              precision    recall  f1-s

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4976
           1       0.14      0.01      0.01       266

    accuracy                           0.95      5242
   macro avg       0.55      0.50      0.49      5242
weighted avg       0.91      0.95      0.92      5242

_______________________________________________________________


FEATURES: buildings

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4952
           1       0.15      0.01      0.01       290

    accuracy                           0.94      5242
   macro avg       0.55      0.50      0.49      5242
weighted avg       0.90      0.94      0.92      5242

_______________________________________________________________


FEATURES: electricity

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5137
           1       0.00      0.00      0.00       105

    ac

### 6. Improve your model
Use grid search to find better parameters. 

In [84]:
# dsiplaying all the parameters 

model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x00000201C7B65268>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_fea

In [85]:
parameters = {'vect__ngram_range':((1, 1), (1, 2)),
              'vect__max_features': (None, 5000, 10000),
              'clf__estimator__max_depth': [10,30,20,50, None],
              'clf__estimator__min_samples_leaf':[2, 7, 10],
              'clf__estimator__min_samples_split': [2, 3, 4]
             }
cv= GridSearchCV(model, parameters)
cv.fit(X_train,y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [86]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
y_pre = pd.DataFrame(y_pred, columns = y_test.columns)
for i in y_test.columns:
    print('_______________________________________________________________\n\n')
    print('FEATURES: {}\n'.format(i))
    print(classification_report(y_test[i],y_pre[i]))

_______________________________________________________________


FEATURES: related

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1247
           1       0.75      1.00      0.86      3953
           2       0.00      0.00      0.00        42

    accuracy                           0.75      5242
   macro avg       0.25      0.33      0.29      5242
weighted avg       0.57      0.75      0.65      5242

_______________________________________________________________


FEATURES: request

              precision    recall  f1-score   support

           0       0.82      1.00      0.90      4310
           1       0.00      0.00      0.00       932

    accuracy                           0.82      5242
   macro avg       0.41      0.50      0.45      5242
weighted avg       0.68      0.82      0.74      5242

_______________________________________________________________


FEATURES: offer

              precision    recall  f1-s

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4952
           1       0.00      0.00      0.00       290

    accuracy                           0.94      5242
   macro avg       0.47      0.50      0.49      5242
weighted avg       0.89      0.94      0.92      5242

_______________________________________________________________


FEATURES: electricity

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5137
           1       0.00      0.00      0.00       105

    accuracy                           0.98      5242
   macro avg       0.49      0.50      0.49      5242
weighted avg       0.96      0.98      0.97      5242

_______________________________________________________________


FEATURES: tools

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5205
           1       0.00      0.00      0.00        37

    accura

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
def new_model():
    modelclassifier=MultiOutputClassifier(AdaBoostClassifier())
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', modelclassifier)
    ])
    return pipeline

In [11]:
n_model = new_model()
n_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x0000018845611D08>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMM

In [12]:
y_pred = n_model.predict(X_test)
Accuracy_overall=(y_pred == y_test).mean().mean()
print('Avg Accuracy Overall {0:.3f}% \n'.format(Accuracy_overall*100))

Avg Accuracy Overall 92.652% 



In [13]:
n_model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000018845611D08>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                      base_estimator=None,
                                                      learning_rate=1.0,
                                                      n_estimators=50,
                                                      random_state=None)

In [14]:
 parameters = {'vect__ngram_range':((1, 1), (1, 2)),
              'vect__max_features': (None, 5000, 10000,20000),
              'tfidf__use_idf': (True, False),
              'clf__estimator__learning_rate': [0.1,0.2,0.3],
              'clf__estimator__random_state':[10,20, 4,None]}
             

c_v = GridSearchCV(n_model,parameters)

In [15]:
c_v.fit(X_train, y_train)
y_pred = c_v.predict(X_test)
y_pre = pd.DataFrame(y_pred, columns = y_test.columns)
for i in y_test.columns:
    print('_______________________________________________________________\n\n')
    print('FEATURES: {}\n'.format(i))
    print(classification_report(y_test[i],y_pre[i]))

_______________________________________________________________


FEATURES: related

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1202
           1       0.76      1.00      0.87      4005
           2       0.00      0.00      0.00        35

    accuracy                           0.76      5242
   macro avg       0.25      0.33      0.29      5242
weighted avg       0.58      0.76      0.66      5242

_______________________________________________________________


FEATURES: request

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      4380
           1       0.00      0.00      0.00       862

    accuracy                           0.84      5242
   macro avg       0.42      0.50      0.46      5242
weighted avg       0.70      0.84      0.76      5242

_______________________________________________________________


FEATURES: offer

              precision    recall  f1-s

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4979
           1       0.00      0.00      0.00       263

    accuracy                           0.95      5242
   macro avg       0.47      0.50      0.49      5242
weighted avg       0.90      0.95      0.93      5242

_______________________________________________________________


FEATURES: electricity

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5123
           1       0.00      0.00      0.00       119

    accuracy                           0.98      5242
   macro avg       0.49      0.50      0.49      5242
weighted avg       0.96      0.98      0.97      5242

_______________________________________________________________


FEATURES: tools

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5204
           1       0.00      0.00      0.00        38

    accura

### 9. Export your model as a pickle file

In [26]:
Pkl_n_model = 'n_model.pkl'  

with open(Pkl_n_model, 'wb') as file:  
    pickle.dump(n_model, file)

In [27]:
# Load the Model back from file
with open(Pkl_n_model, 'rb') as file:  
    Pkl_n_model = pickle.load(file)

Pkl_n_model

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x0000018845611D08>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMM

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.